In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

from dask_gateway import Gateway
from dask.distributed import Client

gateway = Gateway()
cluster = gateway.new_cluster()

cluster.scale(10)
client = Client(cluster,timeout="120s")
cluster

AttributeError: 'VBox' object has no attribute '_ipython_display_'

GatewayCluster<prod.fbea67fc6949418faea130e52994d8ae, status=running>

In [2]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

#model_names=['CanESM5','CESM2',\
#             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
#                 'FIO-ESM-2-0','IPSL-CM6A-LR',\
#                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

model_names=['HadGEM3-GC31-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp585"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    lonO=theta_pi.longitude
    latO=theta_pi.latitude
#    lonO_b=theta_pi.lon_bnds
#    latO_b=theta_pi.lat_bnds

    grid_in = {'lon': lonO, 'lat': latO} 

    grid_out = xr.Dataset({'lat': (['lat'], np.arange(-5, 5, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})
                       
    regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
 
    thetao_regrid = regridder(theta_pi)
    
    theta_upper=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(0,75))
    theta_lower=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(90,110))
    weights = thetao_regrid.lev
    ocean_weighted1 = theta_upper.weighted(weights)
    weighted_upper = ocean_weighted1.mean(('lev'))

    ocean_weighted2 = theta_lower.weighted(weights)
    weighted_lower = ocean_weighted2.mean(('lev'))

    strat=weighted_upper-weighted_lower
    strat.load()
    strat.to_netcdf('strat_'+model_names[x]+'_ssp585_1.nc')
    

In [3]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[1]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

#model_names=['CanESM5','CESM2',\
#             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
#                 'FIO-ESM-2-0','IPSL-CM6A-LR',\
#                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

model_names=['HadGEM3-GC31-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp585"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    lonO=theta_pi.longitude
    latO=theta_pi.latitude
#    lonO_b=theta_pi.lon_bnds
#    latO_b=theta_pi.lat_bnds

    grid_in = {'lon': lonO, 'lat': latO} 

    grid_out = xr.Dataset({'lat': (['lat'], np.arange(-5, 5, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})
                       
    regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
 
    thetao_regrid = regridder(theta_pi)
    
    theta_upper=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(0,75))
    theta_lower=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(90,110))
    weights = thetao_regrid.lev
    ocean_weighted1 = theta_upper.weighted(weights)
    weighted_upper = ocean_weighted1.mean(('lev'))

    ocean_weighted2 = theta_lower.weighted(weights)
    weighted_lower = ocean_weighted2.mean(('lev'))

    strat=weighted_upper-weighted_lower
    strat.load()
    strat.to_netcdf('strat_'+model_names[x]+'_ssp585_2.nc')
    

In [4]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[2]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

#model_names=['CanESM5','CESM2',\
#             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
#                 'FIO-ESM-2-0','IPSL-CM6A-LR',\
#                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

model_names=['HadGEM3-GC31-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp585"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    lonO=theta_pi.longitude
    latO=theta_pi.latitude
#    lonO_b=theta_pi.lon_bnds
#    latO_b=theta_pi.lat_bnds

    grid_in = {'lon': lonO, 'lat': latO} 

    grid_out = xr.Dataset({'lat': (['lat'], np.arange(-5, 5, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})
                       
    regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
 
    thetao_regrid = regridder(theta_pi)
    
    theta_upper=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(0,75))
    theta_lower=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(90,110))
    weights = thetao_regrid.lev
    ocean_weighted1 = theta_upper.weighted(weights)
    weighted_upper = ocean_weighted1.mean(('lev'))

    ocean_weighted2 = theta_lower.weighted(weights)
    weighted_lower = ocean_weighted2.mean(('lev'))

    strat=weighted_upper-weighted_lower
    strat.load()
    strat.to_netcdf('strat_'+model_names[x]+'_ssp585_3.nc')
    

In [5]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

#model_names=['CanESM5','CESM2',\
#             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
#                 'FIO-ESM-2-0','IPSL-CM6A-LR',\
#                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

model_names=['HadGEM3-GC31-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"abrupt-4xCO2"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    lonO=theta_pi.longitude
    latO=theta_pi.latitude
#    lonO_b=theta_pi.lon_bnds
#    latO_b=theta_pi.lat_bnds

    grid_in = {'lon': lonO, 'lat': latO} 

    grid_out = xr.Dataset({'lat': (['lat'], np.arange(-5, 5, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})
                       
    regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
 
    thetao_regrid = regridder(theta_pi)
    
    theta_upper=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(0,75))
    theta_lower=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(90,110))
    weights = thetao_regrid.lev
    ocean_weighted1 = theta_upper.weighted(weights)
    weighted_upper = ocean_weighted1.mean(('lev'))

    ocean_weighted2 = theta_lower.weighted(weights)
    weighted_lower = ocean_weighted2.mean(('lev'))

    strat=weighted_upper-weighted_lower
    strat.load()
    strat.to_netcdf('strat_'+model_names[x]+'_4x.nc')
    

In [6]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar



#model_names=['CanESM5','CESM2',\
#             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
#                 'FIO-ESM-2-0','IPSL-CM6A-LR',\
#                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

model_names=['HadGEM3-GC31-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"piControl"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    lonO=theta_pi.longitude
    latO=theta_pi.latitude
#    lonO_b=theta_pi.lon_bnds
#    latO_b=theta_pi.lat_bnds

    grid_in = {'lon': lonO, 'lat': latO} 

    grid_out = xr.Dataset({'lat': (['lat'], np.arange(-5, 5, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})
                       
    regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
 
    thetao_regrid = regridder(theta_pi)
                       

    theta_upper=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(0,75))
    theta_lower=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(90,110))
    weights = thetao_regrid.lev
    ocean_weighted1 = theta_upper.weighted(weights)
    weighted_upper = ocean_weighted1.mean(('lev'))

    ocean_weighted2 = theta_lower.weighted(weights)
    weighted_lower = ocean_weighted2.mean(('lev'))

    strat=weighted_upper-weighted_lower
    strat.load()
    strat.to_netcdf('strat_'+model_names[x]+'_piControl.nc')
    

In [7]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar



#model_names=['CanESM5','CESM2',\
#             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
#                 'FIO-ESM-2-0','IPSL-CM6A-LR',\
#                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

model_names=['HadGEM3-GC31-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"1pctCO2"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    theta_pi=get_df(theta)
    theta_pi=theta_pi.mean('time')
    
    lonO=theta_pi.longitude
    latO=theta_pi.latitude
#    lonO_b=theta_pi.lon_bnds
#    latO_b=theta_pi.lat_bnds

    grid_in = {'lon': lonO, 'lat': latO} 

    grid_out = xr.Dataset({'lat': (['lat'], np.arange(-5, 5, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})
                       
    regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
 
    thetao_regrid = regridder(theta_pi)
                       

    theta_upper=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(0,75))
    theta_lower=thetao_regrid.sel(lat=slice(-5,5),lon=slice(150,220)).sel(lev=slice(90,110))
    weights = thetao_regrid.lev
    ocean_weighted1 = theta_upper.weighted(weights)
    weighted_upper = ocean_weighted1.mean(('lev'))

    ocean_weighted2 = theta_lower.weighted(weights)
    weighted_lower = ocean_weighted2.mean(('lev'))

    strat=weighted_upper-weighted_lower
    strat.load()
    strat.to_netcdf('strat_'+model_names[x]+'_1pct.nc')
    